## Imports

In [1]:
import copy
import os
import random
import wave
from pathlib import Path
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.data_utils import get_file
from keras.utils.np_utils import to_categorical

# Lab 6: Project CERN

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Download, cache and extract bird song data

In [3]:
dataset_dir = Path('datasets')
remote_dataset_file='/content/drive/My Drive/dataset_v2.tar.gz'
#data_for_processing = keras.utils.get_file(myFile, 'file://'+fullPath)
if not (dataset_dir/'testing_list.txt').exists(): # Assume dataset already downloaded/extracted if testing list is present
    get_file(None, "file://" + remote_dataset_file,
                    extract=True,
                    cache_dir='.',
                    cache_subdir=dataset_dir)

## Load Bird song data

In [4]:
# Classes to handle, ordered by label
CLASSES = ['alauda_arvensis', 'cuculus_canorus', 'delichon_urbicum', 'emberiza_cirlus', 'emberiza_citrinella', 'falco_tinnunculus', 'muscicapa_striata', 'sylvia_borin', 'tachybaptus_ruficollis', 'tyto_alba']

with (dataset_dir/'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob(f'**/*.wav'):
    if not recording.parent.name in CLASSES: # Ignore unused classes
        print("avoid")
        continue
    label = CLASSES.index(recording.parent.name) # Assign class number
    
    with wave.open(str(recording)) as f: # Read wave file
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy() # As 16-bit signed integer
        
    data = data.astype(np.float32) # Convert to 32-bit floating-point
    data.resize((16000, 1)) # Resize to 1s (16kHz) with zero-padding, 1 channel

    if str(recording.relative_to(dataset_dir)) in testing_list: # Assign to test set if file in test list
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Normalize data

In [5]:
# Use train set as a reference for mean/std
x_mean = x_train.mean()
x_std = x_train.std()

x_train -= x_mean
x_test  -= x_mean
x_train /= x_std
x_test  /= x_std

## Export small dataset (250 random vectors)

In [6]:
perms = np.random.permutation(len(y_test))[0:250]
x_test_250 = x_test[perms]
y_test_250 = y_test[perms]
np.savetxt('x_test_gsc_250.csv', x_test_250.reshape((x_test_250.shape[0], -1)), delimiter=',', fmt='%s')
np.savetxt('y_test_gsc_250.csv', y_test_250, delimiter=',', fmt='%s')

## Build model M5

In [7]:
model = Sequential()
model.add(Input(shape=(16000, 1)))

model.add(MaxPool1D(pool_size=10))
model.add(Conv1D(filters=32, kernel_size=80, strides=4, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=3))

model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d (MaxPooling1D  (None, 1600, 1)          0         
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 381, 32)           2592      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 95, 32)           0         
 1D)                                                             
                                                                 
 conv1d_1 (Conv1D)           (None, 93, 32)            3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 23, 32)           0         
 1D)                                                             
                                                        

## Train model

In [8]:
model.fit(x_train, y_train, epochs=10, batch_size=150, validation_data=(x_test, y_test))

Epoch 1/10
155/155 [==============================] - 7s 19ms/step - loss: 2.1451 - categorical_accuracy: 0.2409 - val_loss: 2.0335 - val_categorical_accuracy: 0.2884
Epoch 2/10
155/155 [==============================] - 2s 14ms/step - loss: 1.8846 - categorical_accuracy: 0.3632 - val_loss: 1.7948 - val_categorical_accuracy: 0.4004
Epoch 3/10
155/155 [==============================] - 2s 16ms/step - loss: 1.7132 - categorical_accuracy: 0.4272 - val_loss: 1.6266 - val_categorical_accuracy: 0.4605
Epoch 4/10
155/155 [==============================] - 2s 14ms/step - loss: 1.5899 - categorical_accuracy: 0.4683 - val_loss: 1.5129 - val_categorical_accuracy: 0.5068
Epoch 5/10
155/155 [==============================] - 2s 13ms/step - loss: 1.4971 - categorical_accuracy: 0.5031 - val_loss: 1.4601 - val_categorical_accuracy: 0.5290
Epoch 6/10
155/155 [==============================] - 2s 13ms/step - loss: 1.4295 - categorical_accuracy: 0.5276 - val_loss: 1.4289 - val_categorical_accuracy: 0.528

## Evaluate model on test dataset

In [9]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

312/312 - 1s - loss: 1.3118 - categorical_accuracy: 0.5828 - 1s/epoch - 5ms/step
312/312 [==============================] - 1s 3ms/step
tf.Tensor(
[[625   5  45  81  38  19  25  56  54  12]
 [  7 661  83  63   6  53  48  13  48  12]
 [ 50  23 539 120  14  62 115  33  36  31]
 [ 38   1  80 606  35  43  33  23  95  24]
 [ 41  12  43 144 378 125  47  77  82  30]
 [  8  10  55 100  44 524  42  64 101  70]
 [ 11  37 110 108  31 115 380  20  99  57]
 [ 99   5  39  36  44  81  10 680  72   4]
 [ 15  12  38 103  12  80  54  13 596  34]
 [  9   6  30  76   2  12  44   0  21 820]], shape=(10, 10), dtype=int32)


## Evaluate model on small dataset

In [10]:
model.evaluate(x_test_250, y_test_250, verbose=2)
pred_test_250 = model.predict(x_test_250)
print(tf.math.confusion_matrix(y_test_250.argmax(axis=1), pred_test_250.argmax(axis=1)))

8/8 - 0s - loss: 1.2440 - categorical_accuracy: 0.6040 - 168ms/epoch - 21ms/step
8/8 [==============================] - 0s 5ms/step
tf.Tensor(
[[13  0  2  1  0  0  0  1  3  1]
 [ 0 16  3  1  0  0  1  1  0  1]
 [ 1  0 17  4  0  3  4  2  0  0]
 [ 1  0  0 15  1  2  1  0  2  0]
 [ 0  0  0  5 10  3  0  3  1  0]
 [ 0  1  1  4  1  9  1  2  1  2]
 [ 0  1  5  3  0  2 15  1  2  2]
 [ 2  0  1  0  4  1  1 19  2  0]
 [ 0  1  2  3  0  1  1  0 17  1]
 [ 0  0  0  1  0  0  1  0  0 20]], shape=(10, 10), dtype=int32)


## Save trained model

In [11]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [12]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [13]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Generate C code for the trained model with 16-bit fixed-point representation

In [14]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

———————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                           | Layer                            | Outputs                         
———————————————————————————————————————————————————————————————————————————————————————————————————————
                                 | input_1                          | max_pooling1d                   
-------------------------------------------------------------------------------------------------------
input_1                          | max_pooling1d                    | conv1d                          
-------------------------------------------------------------------------------------------------------
max_pooling1d                    | conv1d                           | max_pooling1d_1                 
-------------------------------------------------------------------------------------------------------
conv1d                           | max_pooling1d_1                  

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [16]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp 
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv

g++: error: main.cpp: No such file or directory
/bin/bash: ./gsc_fixed: No such file or directory
